**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install yellowbrick
!{sys.executable} -m pip install --quiet sweetviz
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from class_utils.sklearn import (
    make_ext_column_transformer, transformer_extensions
)
from sklearn.cluster import KMeans

import sweetviz as sv
from class_utils.plots import crosstab_plot, ColGrid, RainCloud
import seaborn as sns

from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
# revert yellowbrick's invasive changes to matplotlib's
# styling; also suppressing deprecation warnings
import warnings
import yellowbrick

with warnings.catch_warnings(record=True) as w:
    yellowbrick.style.rcmod.set_aesthetic('reset')
    yellowbrick.style.rcmod.reset_orig()
    
cluster_colors = sns.color_palette()

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("Mall_Customers.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Segmentácia zákazníkov pomocou zhlukovania

Ako ďalší príklad sa pozrieme na dátovú množinu zákazníkov nákupného centra a pokúsime sa identifikovať segmenty zákazníkov, t.j. skupiny zákazníkov, ktoré zdieľajú určité spoločné charakteristiky. Poznať segmenty zákazníkov môže byť veľmi užitočné – umožňuje spoločnostiam napr. používať rôzne marketingové stratégie keď cielia na rôzne segmenty atď.

Začnime načítaním dátovej množiny. Ako vidno, nie je príliš zložitá – obsahuje len rod (gender), vek (age), ročný príjem (annual income) a výdavkové skóre (spending score) každého zákazníka. Uvidíme však, že napriek tomu ju stále dokážeme využiť na odhalenie zaujímavých zákonitostí.



In [ ]:
df = pd.read_csv("data/Mall_Customers.csv")
df.head()

### Exploratívna analýza

V prvom kroku nášho procesu budeme na dáta aplikovať ľahkú exploratívnu analýzu. Najprv si môžeme spustiť `sv.analyze`, čím získame základné informácie o typoch a rozdeleniach stĺpcov, ich koreláciách, chýbajúcich dátach a pod.



In [ ]:
report = sv.analyze(df, target_feat='Spending Score (1-100)')
report.show_notebook()

Následne by sme mohli preskúmať vzťahy medzi rôznymi pármi premenných. Napr. by sme mohli použiť husličkové grafy na zobrazenie rozdelení numerických premenných, podmienené rodom zákazníka.



In [ ]:
g = ColGrid(df, 'Gender', ["Age", "Annual Income (k$)", "Spending Score (1-100)"], col_wrap=2)
g.map_dataframe(sns.violinplot);
plt.gcf().set_size_inches(10, 6)

---
### Úloha 1: Vzťahy medzi numerickými premennými a výdavkovým skóre

**Aby sme preskúmali závislosti medzi rozličnými numerickými premennými a výdavkovým skóre, vytvorte mriežku bodových diagramov s týmito *numerickými premennými*  na horizontálnej osi a *výdavkovým skóre*  na vertikálnej osi.** 

---


In [ ]:
g = ColGrid(      # ---
    
# ---

---
#### Úloha 2: Opis pozorovaných segmentov

Keď ste teraz zobrazili a vizuálne preskúmali bodové diagramy, mali by ste pozorovať 2 zhluky v grafe *age*  vs. *spending score*  a 5 zhlukov v grafe *annual income*  vs. *spending score* . V nasledujúcej bunke **opíšte, čomu by každý tento segment zákazníkov mohol zodpovedať – ako by sa dal interpretovať** .

---


### Predspracovanie

Keď sme teraz realizovali základnú exploráciu, povedzme, že by sme teraz chceli získať určitý počet interpretovateľných zhlukov a potom možno ďalej preskúmať vlastnosti každého z nich. Už vyššie ste uvažovali nad a v dobrom prípade ste aj poskytli nejakú interpretáciu piatich zhlukov prítomných v grafe annual income vs. spending score. Skúsme teda teraz zachytiť tieto zhluky.

Za týmto účelom teraz z dátového rámca odstránime všetky stĺpce okrem `Annual Income (k$)` a `Spending Score (1-100)` a na tieto dva zvyšné aplikujeme určité štandardné predspracovanie.



In [ ]:
# all inputs are numeric
categorical_inputs = [
    # "Gender"
]

numeric_inputs = [
    # "Age",
    "Annual Income (k$)", "Spending Score (1-100)"
]

# the preprocessing pipeline
input_preproc = make_ext_column_transformer(
    (make_pipeline(
        transformer_extensions(
            SimpleImputer(strategy='constant', fill_value='MISSING')
        ),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        transformer_extensions(
            SimpleImputer()
        ),
        StandardScaler()),
     numeric_inputs),

    inverse_dropped='ignore',
    verbose_feature_names_out=False
)

# the preprocessed data and the classes
X = input_preproc.fit_transform(df)

---
### Úloha 3: Aplikácia metódy $k$-means na dáta

**Ako ďalšiu úlohu aplikujte na dáta $k$-means zhlukovanie. Výsledné identifikátory zhlukov priraďte do stĺpca `clust` v dátovom rámci `df`.**  Poznámka: Aby nasledujúce bunky správne fungovali, priraďte okrem toho `KMeans` objektu identifikátor `model`.

---


In [ ]:
model = # ---


df["clust"] = # ---


Jednou z užitočných vlastností $k$-means je, že zhluky sú guľovitého tvaru, vďaka čomu sa dajú ľahko reprezentovať ťažiskami a preto sú relatívne dobre interpretovateľné. Keď používame `KMeans` objekt z balíčka scikit-learn, môžeme extrahovať stredy zhlukov pomocou `model.cluster_centers_`.

Tieto stredy zhlukov sú, samozrejme, už štandardizované, čo nemusí byť dobré ak sa ich snažíme interpretovať. Použijeme preto objekt `input_preproc` a transformujeme ich späť do pôvodnej škály (k$ pre ročný príjem a 1-100 pre výdavkové skóre) skôr než ich zobrazíme.



In [ ]:
cluster_centers = input_preproc.inverse_transform(model.cluster_centers_)
cluster_centers

### Analýza objavených zhlukov

Keď sme získali identifikátory zhlukov, môžeme realizovať ďalšie analýzy, aby sme sa dozvedeli viac o zákazníkov patriacich do každého z nich. Keďže sme sa snažili zachytiť 5 zhlukov viditeľných v grafe annual income vs. spending score, najprv sa uistime, že sa nám to korektne podarilo.



In [ ]:
sns.scatterplot(x="Annual Income (k$)", y="Spending Score (1-100)", data=df, s=20, hue="clust", palette=cluster_colors[:cluster_centers.shape[0]])
sns.scatterplot(x="Annual Income (k$)", y="Spending Score (1-100)", data=cluster_centers, s=100, color='k')
plt.grid(ls='--')
plt.gca().set_axisbelow(True)

Ďalej sa môžeme pozrieť na vzťahy medzi číslom zhluku a ďalšími premennými – obdobným spôsobom ako to robíme v rámci exploratívnej analýzy. Zobrazme si husličkové grafy zhlukov vs. trom číselným premenným, ktoré máme.

Jedna vec, ktorú si môžeme všimnúť, je, že rozdelenie veku je v dvoch z týchto zhlukov viacej koncentrované než v ostatných. Napr. v skupine, ktorá má vysoké príjmy a veľa míňa je medián veku 32, minimum 27 a maximum 40. V skupine ktorá míňa veľa napriek tomu, že má nízky príjem, sú veky podstatne nižšie – medián je 23.5 a maximum je 35. Ostatné zhluky pokrývajú viac menej celý vekový rozsah.

Ako vidno, toto nám už hovorí viacero užitočných vecí – ukazuje to napríklad, že v rámci našej vzorky majú starší ľudia nižšiu tendenciu míňať nezodpovedne než mladší ľudia.



In [ ]:
g = ColGrid(df, 'clust', ["Age", "Annual Income (k$)", "Spending Score (1-100)"], col_wrap=2)
g.map_dataframe(sns.violinplot);
plt.gcf().set_size_inches([12, 8])

Môžeme si tiež zobraziť maticu vzťahov medzi rodom a jednotlivými zhlukmi. Čo v tomto prípade môže človeku udrieť do očí je, že v dvoch z našich zhlukov je podstatne menej mužov než žien a že ide o zhluky zodpovedajúce zákazníkom z nízkym príjmom.



In [ ]:
plt.figure(figsize=(5, 5))
crosstab_plot(x='Gender', y='clust', data=df);

Zdá sa, že v rámci našej vzorky majú ženy o niečo vyššiu pravdepodobnosť mať nízky príjem než muži (v nízkopríjmových zhlukoch je o niečo viac žien než mužov). Aby sme to analyzovali podrobnejšie, môžeme sa pozrieť na dáta o zákazníkoch zarábajúcich menej než 40k. Vyfiltrujme si ich a určme početnosť mužov a žien. Z toho naozaj uvidíme, že v danej kategórii je mužov menej než žien.



In [ ]:
df_low = df[df["Annual Income (k$)"] < 40]
df_low_male = df_low[df_low["Gender"] == "Male"]
df_low_female = df_low[df_low["Gender"] == "Female"]

print(
    f"Number of males with <40k income: {len(df_low_male)};\n"
    f"Number of females with <40k income: {len(df_low_female)};\n"
    f"The ratio of males vs. females is: {len(df_low_male) / len(df_low_female)}"
)

Môžeme si dokonca zobraziť aj raincloud graf, aby sme získali úplnejšiu predstavu o rozdelení medzi mužmi/ženami v rámci tohto rozsahu príjmov.



In [ ]:
RainCloud(x="Gender", y="Annual Income (k$)", data=df_low)